In [1]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import pycountry
from geojson import FeatureCollection, dump

In [18]:
load_dotenv()
TOKEN=os.getenv("MAPBOX_TOKEN")
df = pd.read_csv("../output/processed/clean.csv")
origins = list(df.origin.unique())
destinations = list(df.destination.unique())

In [19]:
in_first = set(origins)
in_second = set(destinations)
in_second_but_not_in_first = in_second - in_first
countries = origins + list(in_second_but_not_in_first)

In [20]:
df = pd.DataFrame({"country":countries})

In [21]:
headers = {'Accept': 'application/json'}
lst = []
for i in countries:
    obj = {}
    obj["type"]="Feature"
    obj["properties"]={"name": pycountry.countries.get(alpha_2=i).name}
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{i}.json?&types=country&access_token={TOKEN}"
    r = requests.get(url)
    jason = r.json()
    obj["properties"]["code"] = i.lower()
    coords = jason["features"][0]["center"]
    obj["geometry"] = {"type":"Point", "coordinates": coords}
    lst.append(obj)
    df.loc[df["country"] == i, "lat"]=coords[0]
    df.loc[df["country"] == i, "lon"]=coords[1]

In [22]:
with open('../output/processed/countries.geojson', 'w') as f:
    dump(FeatureCollection(lst), f)

In [23]:
df

,country,lat,lon
0,ad,1.576766,42.548654
1,au,134.489563,-25.734968
2,by,28.046788,53.539998
3,ba,50.559644,26.022407
4,br,-51.619789,-9.588903
...,...,...,...
150,gi,-11.029862,10.785546
151,us,-97.922211,39.381266
152,et,39.616032,8.626220
153,tj,71.042004,38.838508


In [24]:
df.to_csv("../output/processed/coords.csv", index=False)